# Setup

## Imports

In [ ]:
!pip install torch transformers datasets

In [2]:
!git config --global credential.helper store

In [3]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, OPTForSequenceClassification
from transformers import AutoTokenizer, AlbertForSequenceClassification
from transformers import AutoTokenizer, GPT2ForSequenceClassification
import time
from sklearn.metrics import classification_report

2025-04-25 20:39:43.604987: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745613583.806026      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745613583.866153      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Dataset and Models

In [4]:
dataset = load_dataset("/kaggle/input/sentiment/sentiment140.py", name="sentiment140")

The repository for sentiment140 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sentiment140.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
from huggingface_hub import login
token = ""

In [9]:
student_name = "facebook/opt-125m"
teacher_name = "facebook/opt-350m"
num_labels = 2
student_tokenizer = AutoTokenizer.from_pretrained(student_name, token=token)
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_name, token=token)
student_model = OPTForSequenceClassification.from_pretrained(student_name, num_labels=num_labels, token=token)
teacher_model = OPTForSequenceClassification.from_pretrained(teacher_name, num_labels=num_labels, token=token)

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

...

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-125m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/251M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def map_labels(example):
    example["sentiment"] = 0 if example["sentiment"] == 0 else 1
    return example

dataset = dataset.map(map_labels)

Map:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

DOWNSAMPLING FOR TRAINING

In [11]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd

# Step 1: Convert to pandas
df = dataset['train'].to_pandas()

_, df_balanced = train_test_split(
    df,
    stratify=df["sentiment"],
    test_size=0.1,
    random_state=42
)

# train_dataset_small = Dataset.from_pandas(df_balanced)
# train_dataset_small = train_dataset_small.remove_columns(["__index_level_0__"])
print(len(df_balanced))

# Step 2: Stratified split (70/30)
df_train, df_temp = train_test_split(
    df_balanced,
    stratify=df_balanced["sentiment"],
    test_size=0.3,
    random_state=42
)

# Step 3: Split remaining 30% into val/test (50/50)
df_val, df_test = train_test_split(
    df_temp,
    stratify=df_temp["sentiment"],
    test_size=0.5,
    random_state=42
)

print(len(df_train))
print(len(df_val))
print(len(df_test))

# Step 4: Convert back to Hugging Face Datasets
train_dataset = Dataset.from_pandas(df_train).remove_columns(["__index_level_0__"])
val_dataset = Dataset.from_pandas(df_val).remove_columns(["__index_level_0__"])
test_dataset = Dataset.from_pandas(df_test).remove_columns(["__index_level_0__"])

# Step 5: Tokenization function
def student_tokenize_function(example):
    return student_tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

def teacher_tokenize_function(example):
    return teacher_tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

# Step 6: Tokenize each split
student_train_dataset = train_dataset.map(student_tokenize_function, batched=True)
student_val_dataset   = val_dataset.map(student_tokenize_function, batched=True)
student_test_dataset  = test_dataset.map(student_tokenize_function, batched=True)

teacher_train_dataset = train_dataset.map(teacher_tokenize_function, batched=True)
teacher_val_dataset   = val_dataset.map(teacher_tokenize_function, batched=True)
teacher_test_dataset  = test_dataset.map(teacher_tokenize_function, batched=True)

# Step 7: Set PyTorch format
student_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "sentiment"])
student_val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "sentiment"])
student_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "sentiment"])
teacher_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "sentiment"])
teacher_val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "sentiment"])
teacher_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "sentiment"])

# (Optional) Wrap in a DatasetDict
student_dataset = DatasetDict({
    "train": student_train_dataset,
    "validation": student_val_dataset,
    "test": student_test_dataset
})
teacher_dataset = DatasetDict({
    "train": teacher_train_dataset,
    "validation": teacher_val_dataset,
    "test": teacher_test_dataset
})

# Print sizes to confirm
print(f"Train:     {len(student_dataset['train'])}")
print(f"Validation:{len(student_dataset['validation'])}")
print(f"Test:      {len(student_dataset['test'])}")

160000
112000
24000
24000


Map:   0%|          | 0/112000 [00:00<?, ? examples/s]

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/112000 [00:00<?, ? examples/s]

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Train:     112000
Validation:24000
Test:      24000


In [13]:
dataset = student_dataset
train_loader = DataLoader(dataset['train'], batch_size=16, shuffle=True)
val_loader = DataLoader(dataset['validation'], batch_size=16, shuffle=True)
test_loader = DataLoader(dataset['test'], batch_size=16)

teacher_train_loader = DataLoader(teacher_dataset['train'], batch_size=16, shuffle=True)
teacher_val_loader = DataLoader(teacher_dataset['validation'], batch_size=16, shuffle=True)
teacher_test_loader = DataLoader(teacher_dataset['test'], batch_size=16)

In [ ]:
torch.cuda.device_count()

In [14]:
# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
checkpoint = torch.load("/kaggle/working/student.pth", weights_only=True)
student_model.load_state_dict(checkpoint)
checkpoint = torch.load("/kaggle/working/teacher1.pth", weights_only=True)
teacher_model.load_state_dict(checkpoint)
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']
student_optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, student_model.parameters()),
    lr=5e-5,
    weight_decay=0.01
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# student_model.to(device)

# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
teacher_optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, teacher_model.parameters()),
    lr=5e-5,
    weight_decay=0.01
)
# teacher_model.to(device)

# Model Training and Evaluation

## Model Dry Run

In [ ]:
import time
import torch
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

inference_start = time.time()

student_model.eval()
total_correct = 0
total_samples = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["sentiment"].to(device)

        outputs = student_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        total_correct += (predictions == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples
f1_macro = f1_score(all_labels, all_preds, average="macro")
f1_weighted = f1_score(all_labels, all_preds, average="weighted")
inference_time = time.time() - inference_start

print(f"\nTest Accuracy   : {accuracy:.4f}")
print(f"F1 Score (macro): {f1_macro:.4f}")
print(f"F1 Score (weighted): {f1_weighted:.4f}")
print(f"Inference Time  : {inference_time:.2f}s")
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Negative", "Positive"]))

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

## Distillation

### Teacher VS Student raw

In [ ]:
import time
import torch
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

inference_start = time.time()

def train(model, optimizer, train_loader, val_loader):
    
    num_epochs = 2
    best_val_f1 = 0.0
    best_model_state = None
    
    total_train_start = time.time()
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
    
        for batch in tqdm(train_loader):
            optimizer.zero_grad()
    
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["sentiment"].to(device)
    
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
    
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
    
        avg_loss = running_loss / len(train_loader)
        print(f"\nEpoch {epoch+1}/{num_epochs} - Training Loss: {avg_loss:.4f}")
    
        model.eval()
        all_preds = []
        all_labels = []
    
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["sentiment"].to(device)
    
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=1)
    
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
    
        # Metrics
        val_accuracy = accuracy_score(all_labels, all_preds)
        val_f1 = f1_score(all_labels, all_preds, average="macro")
        print(f"Validation Accuracy: {val_accuracy:.4f} - F1 (macro): {val_f1:.4f}")
    
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_state = model.state_dict()

    total_train_time = time.time() - total_train_start
    print(f"\nTotal Training Time: {total_train_time:.2f}s")
    return best_model_state

# student_state = train(student_model, student_optimizer, train_loader, val_loader)
# torch.save(student_state, "/kaggle/working/student.pth")
teacher_state = train(teacher_model.to(device), teacher_optimizer, teacher_train_loader, teacher_val_loader)
torch.save(teacher_state, "/kaggle/working/teacher2.pth")

## Knowledge Distillation

In [ ]:
import time
import torch
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

inference_start = time.time()

def distill(student, teacher, optimizer, train_loader, val_loader, T, soft_target_loss_weight, ce_loss_weight):
    ce_loss = torch.nn.CrossEntropyLoss()
    num_epochs = 4
    best_val_f1 = 0.0
    best_model_state = None
    
    total_train_start = time.time()
    
    for epoch in range(num_epochs):
        teacher.eval()  # Teacher set to evaluation mode
        student.train() # Student to train mode
        running_loss = 0.0
    
        for batch in tqdm(train_loader):
            optimizer.zero_grad()
    
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["sentiment"].to(device)

            with torch.no_grad():
                teacher_logits = teacher(input_ids=input_ids, attention_mask=attention_mask, labels=labels).logits

            # Forward pass with the student model
            student_logits = student(input_ids=input_ids, attention_mask=attention_mask, labels=labels).logits


            soft_targets = torch.nn.functional.softmax(teacher_logits / T, dim=-1)
            soft_prob = torch.nn.functional.log_softmax(student_logits / T, dim=-1)

            # Calculate the soft targets loss. Scaled by T**2 as suggested by the authors of the paper "Distilling the knowledge in a neural network"
            soft_targets_loss = torch.sum(soft_targets * (soft_targets.log() - soft_prob)) / soft_prob.size()[0] * (T**2)

            # Calculate the true label loss
            label_loss = ce_loss(student_logits, labels)

            # Weighted sum of the two losses
            loss = soft_target_loss_weight * soft_targets_loss + ce_loss_weight * label_loss
    
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
    
        avg_loss = running_loss / len(train_loader)
        print(f"\nEpoch {epoch+1}/{num_epochs} - Training Loss: {avg_loss:.4f}")
    
        # Validation
        student.eval()
        all_preds = []
        all_labels = []
    
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["sentiment"].to(device)
    
                outputs = student(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=1)
    
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
    
        # Metrics
        val_accuracy = accuracy_score(all_labels, all_preds)
        val_f1 = f1_score(all_labels, all_preds, average="macro")
        print(f"Validation Accuracy: {val_accuracy:.4f} - F1 (macro): {val_f1:.4f}")
    
        # 💾 Save best model (optional)
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_state = student.state_dict()  # or `copy.deepcopy(model.state_dict())`

    total_train_time = time.time() - total_train_start
    print(f"\nTotal Training Time: {total_train_time:.2f}s")
    return best_model_state

# checkpoint = torch.load("/kaggle/working/student.pth", weights_only=True)
# student_model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']
# student_model = torch.load("/kaggle/working/student.pth", weights_only=True)
# teacher_model = torch.load("/kaggle/working/teacher1.pth", weights_only=True)
distill(
    student_model.to(device),
    teacher_model.to(device),
    student_optimizer,
    train_loader,
    val_loader,
    T=2,
    soft_target_loss_weight=0.25,
    ce_loss_weight=0.75
)

### Produce matrix and graphs

In [ ]:
student_model.eval()
all_preds = []
all_labels = []
inference_start = time.time()
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["sentiment"].to(device)

        outputs = student_model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Metrics
accuracy = accuracy_score(all_labels, all_preds)
f1_macro = f1_score(all_labels, all_preds, average="macro")
f1_weighted = f1_score(all_labels, all_preds, average="weighted")
inference_time = time.time() - inference_start

print(f"\nTest Accuracy   : {accuracy:.4f}")
print(f"F1 Score (macro): {f1_macro:.4f}")
print(f"F1 Score (weighted): {f1_weighted:.4f}")
print(f"Inference Time  : {inference_time:.2f}s")
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Negative", "Positive"]))

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# LORA (for large teacher training)

In [ ]:
!pip install peft

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=16,                              # LoRA rank (8 or 16 are common)
    lora_alpha=32,                     # Scaling factor
    lora_dropout=0.1,                  # Dropout for LoRA layers
    bias="none",                       # Don't train bias terms
    task_type=TaskType.SEQ_CLS,        # Task type: sequence classification
    target_modules=["q_proj", "v_proj"]  # or ["c_attn"] for GPT-2
)


model = get_peft_model(student_model, lora_config)
model.print_trainable_parameters()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

num_epochs = 6
best_val_f1 = 0.0
best_model_state = None

total_train_start = time.time()
optimizer = teacher_optimizer
model = teacher_model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch in tqdm(train_loader):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["sentiment"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"\nEpoch {epoch+1}/{num_epochs} - Training Loss: {avg_loss:.4f}")

    # Validation
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["sentiment"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Metrics
    val_accuracy = accuracy_score(all_labels, all_preds)
    val_f1 = f1_score(all_labels, all_preds, average="macro")
    print(f"Validation Accuracy: {val_accuracy:.4f} - F1 (macro): {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_model_state = model.state_dict()  # or `copy.deepcopy(model.state_dict())`

total_train_time = time.time() - total_train_start
print(f"\nTotal Training Time: {total_train_time:.2f}s")

# (Optional) Load best model
# model.load_state_dict(best_model_state)

### Test Evaluation

In [ ]:
import time
import torch
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

inference_start = time.time()

model.eval()
total_correct = 0
total_samples = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["sentiment"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        total_correct += (predictions == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples
f1_macro = f1_score(all_labels, all_preds, average="macro")
f1_weighted = f1_score(all_labels, all_preds, average="weighted")
inference_time = time.time() - inference_start

print(f"\nTest Accuracy   : {accuracy:.4f}")
print(f"F1 Score (macro): {f1_macro:.4f}")
print(f"F1 Score (weighted): {f1_weighted:.4f}")
print(f"Inference Time  : {inference_time:.2f}s")
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Negative", "Positive"]))

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

### Quickly clean up gpu and cpu memory

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()